In [19]:
import pandas as pd
import numpy as np

import re

import kagglehub
import json

from underthesea import text_normalize as vn_normalize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
path = kagglehub.dataset_download("haitranquangofficial/vietnamese-online-news-dataset")
json_path = path + "/news_dataset.json"
print("Path to dataset file:", json_path)

Path to dataset file: C:\Users\ThienLaptop\.cache\kagglehub\datasets\haitranquangofficial\vietnamese-online-news-dataset\versions\1/news_dataset.json


In [4]:
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
data[:5]

[{'id': 218270,
  'author': '',
  'content': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đư

In [5]:
raw_data = pd.read_json(json_path)
raw_data.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


# Preprocessing

In [6]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r"(https?://\S+|www\.\S+)", "", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"[@#]\w+", "", text)
    text = re.sub(r"[^\w\sÀ-ỹ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

VN_STOPWORDS = set("""
và hoặc nhưng là thì mà của những được bằng với về trong trên dưới từ tới cho các một một số
có không chưa đã đang sẽ nữa nhé ạ à ừ ơ dạ vâng ơi nhỉ hả cái này kia ấy vậy thôi luôn
""".split())

def tokenize_vi(text: str) -> str:
    if not text:
        return ""

    # Text normalization
    text = vn_normalize(text)

    # Text cleaning
    text = clean_text(text)

    # Tokenize
    toks = word_tokenize(text, format="text").split()

    # Remove stopwords and digits
    toks = [t for t in toks if t not in VN_STOPWORDS and not t.isdigit()]
    try:
        lemmas = []
        for pair in lemmatize(" ".join(toks)):
            if isinstance(pair, (list, tuple)) and len(pair) >= 2:
                lemmas.append(pair[1])
            else:
                lemmas.append(str(pair))
        toks = lemmas
    except Exception:
        pass
    return toks

def process_text(text: str) -> str:
    return " ".join(tokenize_vi(text))

In [7]:
# Test
text = raw_data.iloc[0]['content']
print(text)
print(process_text(text))

Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn 

In [8]:
# Remove missing values and duplicates
raw_data = raw_data.dropna()
raw_data = raw_data[~raw_data.duplicated()]

# Sampling
sample_data = raw_data.sample(n=1000, random_state=42)
sample_data.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
176595,8693,Nhóm PV,Tạm dừng giải quyết thủ tục tách thửa một số k...,5,1,laodong,Từ phản ánh của Báo Lao Động: Tây Ninh tạm dừn...,Bất động sản,https://laodong.vn/bat-dong-san/tu-phan-anh-cu...,2022-06-15 18:20:12.444876
85401,115186,"Minh Đức-Thứ hai, ngày 11/07/2022 16:25 GMT+7","Theo Tổng cục Thống kê, lực lượng lao động, số...",0,1,vtv.vn,Số người thất nghiệp trong độ tuổi lao động gi...,,https://vtv.vn/xa-hoi/so-nguoi-that-nghiep-tro...,2022-07-11 16:32:01.283629
121681,71252,PHẠM DŨNG,"Chiều 1-6, Bộ Công an tổ chức lễ công bố quyết...",2,1,nld,Ông Nguyễn Sỹ Quang được thăng hàm Thiếu tướng,Trong nước,https://nld.com.vn/thoi-su/ong-nguyen-sy-quang...,2022-07-01 16:32:42.540352
177060,8210,Hoàng Lam,"Theo nội dung video thì, khoảng 13h45 ngày 12/...",4,1,tienphong,"Phá cửa, bắt quả tang vợ ‘tòm tem’ cùng chủ tị...",Xã hội,https://tienphong.vn/pha-cua-bat-qua-tang-vo-t...,2022-06-15 11:20:07.656680
55979,149874,Mai Hà,"Trước đó, Thanh tra Bộ TT-TT đã có kết luận số...",1,1,thanhnien.vn,Tạp chí Kinh doanh và Biên mậu bị phạt 70 triệ...,Thời sự,https://thanhnien.vn/tap-chi-kinh-doanh-va-bie...,2022-07-18 21:03:25.424052


In [9]:
sample_data["processed_text"] = sample_data["content"].apply(process_text)
print(sample_data[["content", "processed_text"]].head())

                                                  content  \
176595  Tạm dừng giải quyết thủ tục tách thửa một số k...   
85401   Theo Tổng cục Thống kê, lực lượng lao động, số...   
121681  Chiều 1-6, Bộ Công an tổ chức lễ công bố quyết...   
177060  Theo nội dung video thì, khoảng 13h45 ngày 12/...   
55979   Trước đó, Thanh tra Bộ TT-TT đã có kết luận số...   

                                           processed_text  
176595  tạm dừng giải_quyết thủ_tục tách thửa một_số k...  
85401   theo tổng_cục thống_kê lực_lượng lao_động ngườ...  
121681  chiều bộ công_an tổ_chức lễ công_bố quyết_định...  
177060  theo nội_dung video khoảng h45 ngày chồng bà h...  
55979   trước đó thanh_tra bộ tt tt kết_luận kl ttra n...  


In [27]:
processed_data = sample_data["processed_text"]
processed_data

176595    tạm dừng giải_quyết thủ_tục tách thửa một_số k...
85401     theo tổng_cục thống_kê lực_lượng lao_động ngườ...
121681    chiều bộ công_an tổ_chức lễ công_bố quyết_định...
177060    theo nội_dung video khoảng h45 ngày chồng bà h...
55979     trước đó thanh_tra bộ tt tt kết_luận kl ttra n...
                                ...                        
75424                                                      
51484     iran đăng_cai hội_nghị thượng_đỉnh bên nga ira...
95568     vượt qua nhiều đối_thủ tên_tuổi man_city sớm c...
94332     chánh văn_phòng nội_các nhật_bản hirokazu_mats...
28596                                                      
Name: processed_text, Length: 1000, dtype: object

In [28]:
cv_uni  = CountVectorizer(ngram_range=(1,1), min_df=2, max_df=0.9)
cv_bi   = CountVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9)   # BoW bigram
tf_uni  = TfidfVectorizer(ngram_range=(1,1), min_df=2, max_df=0.9, norm="l2")
tf_bi   = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, norm="l2")

X_bow_uni = cv_uni.fit_transform(processed_data)
X_bow_bi  = cv_bi.fit_transform(processed_data)
X_tfidf_uni = tf_uni.fit_transform(processed_data)
X_tfidf_bi  = tf_bi.fit_transform(processed_data)

# Result

In [29]:
def corpus_stats(raw, proc):
    import numpy as np
    raw_len = raw.str.len()
    tok_counts = proc.str.split().apply(len)
    return pd.DataFrame({
        "num_docs":[len(raw)],
        "avg_raw_chars":[raw_len.mean()],
        "avg_tokens":[tok_counts.mean()],
        "median_tokens":[tok_counts.median()],
        "min_tokens":[tok_counts.min()],
        "max_tokens":[tok_counts.max()],
    })

stats_df = corpus_stats(processed_data, processed_data)
print(stats_df.round(2))


   num_docs  avg_raw_chars  avg_tokens  median_tokens  min_tokens  max_tokens
0      1000        1987.62      311.22          268.0           0        1314


In [30]:
def matrix_report(X, name):
    nnz = X.nnz
    shape = X.shape
    sparsity = 1 - nnz/(shape[0]*shape[1])
    return pd.Series({
        "representation": name,
        "num_docs": shape[0],
        "vocab_size": shape[1],
        "nonzeros": nnz,
        "sparsity": round(sparsity, 4)
    })

rep_df = pd.concat([
    matrix_report(X_bow_uni,  "BoW unigram"),
    matrix_report(X_bow_bi,   "BoW uni+bi"),
    matrix_report(X_tfidf_uni,"TF-IDF unigram"),
    matrix_report(X_tfidf_bi, "TF-IDF uni+bi"),
], axis=1).T
print(rep_df)


   representation num_docs vocab_size nonzeros sparsity
0     BoW unigram     1000       8968   153892   0.9828
1      BoW uni+bi     1000      38571   262254   0.9932
2  TF-IDF unigram     1000       8968   153892   0.9828
3   TF-IDF uni+bi     1000      38571   262254   0.9932


In [31]:
def top_terms_count(cv, X, k=20):
    vocab = np.array(cv.get_feature_names_out())
    freqs = np.asarray(X.sum(axis=0)).ravel()
    idx = freqs.argsort()[::-1][:k]
    return pd.DataFrame({"term": vocab[idx], "freq": freqs[idx].astype(int)})

def top_terms_tfidf(tfv, X, k=20):
    vocab = np.array(tfv.get_feature_names_out())
    # Lấy trọng số TF-IDF cao nhất trên toàn tập
    max_w = X.max(axis=0).toarray().ravel()
    idx = max_w.argsort()[::-1][:k]
    return pd.DataFrame({"term": vocab[idx], "max_tfidf": np.round(max_w[idx], 4)})

top_bow_uni = top_terms_count(cv_uni, X_bow_uni, 20)
top_bow_bi  = top_terms_count(cv_bi,  X_bow_bi,  20)
top_tf_uni  = top_terms_tfidf(tf_uni, X_tfidf_uni, 20)
top_tf_bi   = top_terms_tfidf(tf_bi,  X_tfidf_bi,  20)

for name, df in [("BoW unigram", top_bow_uni),
                 ("BoW uni+bi",  top_bow_bi),
                 ("TF-IDF unigram", top_tf_uni),
                 ("TF-IDF uni+bi",  top_tf_bi)]:
    print("\n===", name, "===")
    print(df.to_string(index=False))



=== BoW unigram ===
 term  freq
người  2838
   để  2543
  khi  2455
  đến  2119
   đó  1945
 cũng  1920
nhiều  1907
  năm  1806
  vào  1746
  tại  1641
 theo  1527
   ra  1500
 ngày  1458
  sau  1327
  hơn  1270
  ông  1231
 việc  1201
 phải  1197
  như  1178
  tôi  1173

=== BoW uni+bi ===
 term  freq
người  2838
   để  2543
  khi  2455
  đến  2119
   đó  1945
 cũng  1920
nhiều  1907
  năm  1806
  vào  1746
  tại  1641
 theo  1527
   ra  1500
 ngày  1458
  sau  1327
  hơn  1270
  ông  1231
 việc  1201
 phải  1197
  như  1178
  tôi  1173

=== TF-IDF unigram ===
     term  max_tfidf
    nguồn     1.0000
      câu     1.0000
     theo     1.0000
    ttxvn     0.9717
     tiêm     0.8520
      chó     0.8231
     liều     0.8088
   đáp_án     0.8071
      flc     0.7957
dương_vật     0.7731
   hoa_kỳ     0.7722
     lisa     0.7695
     thép     0.7663
      đất     0.7655
      mía     0.7619
       em     0.7587
      thi     0.7551
      sgk     0.7534
       vỏ     0.7465
    messi  